# Обучение Wake Word модели в Google Colab

Этот ноутбук поможет вам обучить собственную модель пробуждения (wake word) используя openwakeword.

## 1. Проверка GPU

Убедитесь, что GPU включён: **Runtime → Change runtime type → GPU**

In [ ]:
# Проверяем доступность GPU
!nvidia-smi

## 2. Клонирование репозитория

In [ ]:
# Клонируем репозиторий
!git clone https://github.com/YOUR_USERNAME/train-wake-word-model.git
%cd train-wake-word-model

## 3. Настройка окружения

Используем скрипт setup.sh для автоматической установки всех зависимостей:

In [ ]:
# Запускаем setup.sh с флагом --no-venv для Colab
# (в Colab не нужно виртуальное окружение)
!bash scripts/setup.sh --no-venv --python python3

print("\n✅ Установка завершена!")

In [ ]:
import openwakeword
import os
import urllib.request

# Определяем директорию для моделей
oww_models_dir = os.path.join(os.path.dirname(openwakeword.__file__), 'resources', 'models')
os.makedirs(oww_models_dir, exist_ok=True)

base_url = "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1"
model_files = [
    'embedding_model.onnx',
    'embedding_model.tflite',
    'melspectrogram.onnx',
    'melspectrogram.tflite'
]

for model_file in model_files:
    dest_path = os.path.join(oww_models_dir, model_file)
    if not os.path.exists(dest_path):
        print(f"Скачивание {model_file}...")
        url = f"{base_url}/{model_file}"
        urllib.request.urlretrieve(url, dest_path)
    else:
        print(f"✓ {model_file} уже существует")

print("\n✅ Модели OpenWakeWord загружены!")

In [ ]:
# Скачиваем базовые данные (MIT RIRs + AudioSet)
!python download_data.py --data-dir ./data --audioset-clips 3000

# Опционально: скачать FMA для музыкального фона (~7 ГБ, займёт дополнительное время)
# !python download_data.py --include-fma --fma-hours 1

## 4. Скачивание данных

Скачаем фоновый шум и импульсные характеристики помещений:

In [ ]:
import yaml
import os

# ============================================
# НАСТРОЙТЕ ЭТИ ПАРАМЕТРЫ ПОД СВОЮ МОДЕЛЬ
# ============================================

config = {
    'model_name': 'my_wake_word',  # 🔧 Измените на своё имя модели
    'target_phrase': ['hey assistant'],  # 🔧 Ваше пробуждающее слово/фраза
    
    # Опционально: добавьте кастомные негативные фразы
    'custom_negative_phrases': [],
    
    # Количество примеров (уменьшено для Colab)
    'n_samples': 3000,
    'n_samples_val': 1000,
    
    # Размеры батчей
    'tts_batch_size': 32,
    'augmentation_batch_size': 16,
    
    # Пути (не изменяйте, если не знаете что делаете)
    'piper_sample_generator_path': './piper-sample-generator',
    'piper_model_path': './piper-sample-generator/models/en_US-libritts_r-medium.pt',
    'output_dir': './my_custom_model',
    'rir_paths': ['./data/mit_rirs'],
    'background_paths': ['./data/audioset_16k'],
    'background_paths_duplication_rate': [1],
    'false_positive_validation_data_path': './data/validation_set_features.npy',
    'augmentation_rounds': 1,
    
    'feature_data_files': {
        'ACAV100M_sample': './data/openwakeword_features_ACAV100M_2000_hrs_16bit.npy'
    },
    'batch_n_per_class': {
        'ACAV100M_sample': 1024,
        'adversarial_negative': 50,
        'positive': 50
    },
    
    # Параметры модели
    'model_type': 'dnn',
    'layer_size': 32,  # Увеличьте до 64 или 128 для лучшего качества
    
    # Параметры обучения (уменьшено для Colab)
    'steps': 5000,  # Увеличьте до 10000+ для лучшего качества
    'max_negative_weight': 1500,
    'target_false_positives_per_hour': 0.2,
    'target_accuracy': 0.8,
    'target_recall': 0.6
}

# Создаём директорию для конфига если не существует
os.makedirs('./config', exist_ok=True)

# Сохраняем конфигурацию
config_path = './config/colab_config.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f)

print(f"✅ Конфигурация создана: {config_path}")
print(f"\nМодель: {config['model_name']}")
print(f"Фраза: {config['target_phrase']}")
print(f"Примеры: {config['n_samples']} обучающих, {config['n_samples_val']} валидационных")
print(f"Шагов обучения: {config['steps']}")

## 5. Настройка конфигурации

Создадим файл конфигурации для обучения:

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --generate_clips

## 6. Генерация синтетических образцов

Создадим позитивные и негативные образцы с помощью TTS:

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --augment_clips

## 7. Аугментация данных

Применим различные аугментации к образцам (шум, реверберация и т.д.):

In [ ]:
!python train.py --training_config ./config/colab_config.yaml --train_model

## 8. Обучение модели

Запустим процесс обучения (может занять 30-90 минут):

In [ ]:
from google.colab import files
import os

# Путь к вашей модели
model_name = config['model_name']
model_path = f"./my_custom_model/{model_name}.onnx"

if os.path.exists(model_path):
    print(f"✅ Модель найдена: {model_path}")
    files.download(model_path)
    print(f"✅ Модель {model_name}.onnx скачана!")
else:
    print(f"❌ Модель не найдена по пути: {model_path}")
    print("\nПроверяем содержимое директории:")
    !ls -la ./my_custom_model/

## 9. Скачивание обученной модели

Скачайте модель на локальный компьютер:

In [ ]:
import openwakeword
from openwakeword.model import Model
import numpy as np

# Загружаем модель
if os.path.exists(model_path):
    owwModel = Model(wakeword_models=[model_path])
    print(f"✅ Модель {model_name} загружена!")
    print(f"\nМодель готова к использованию.")
    print(f"Размер модели: {os.path.getsize(model_path) / 1024:.2f} KB")
else:
    print(f"❌ Модель не найдена: {model_path}")

## 10. Тестирование модели (опционально)

Протестируйте модель на аудио файле:

In [ ]:
from google.colab import drive
import shutil

# Монтируем Google Drive
drive.mount('/content/drive')

# Создаём директорию в Drive
drive_dir = '/content/drive/MyDrive/wake_word_models'
os.makedirs(drive_dir, exist_ok=True)

# Копируем модель
if os.path.exists(model_path):
    shutil.copy(model_path, drive_dir)
    print(f"✅ Модель скопирована в Google Drive: {drive_dir}")
    
    # Опционально: копируем конфигурацию
    shutil.copy(config_path, os.path.join(drive_dir, f"{model_name}_config.yaml"))
    print(f"✅ Конфигурация также сохранена")

## 11. Сохранение в Google Drive (опционально)

Сохраните модель и данные в Google Drive для повторного использования:

In [ ]:
# Раскомментируйте нужные команды для очистки:

# Удалить фоновые данные (освободить ~5-10 ГБ)
# !rm -rf ./data/audioset_16k
# !rm -rf ./data/fma

# Удалить промежуточные аудио файлы (освободить ~2-5 ГБ)
# !rm -rf ./my_custom_model/*/positive_train
# !rm -rf ./my_custom_model/*/negative_train
# !rm -rf ./my_custom_model/*/positive_test
# !rm -rf ./my_custom_model/*/negative_test

# Удалить features файлы (освободить ~1-3 ГБ)
# !rm -rf ./my_custom_model/*/*.npy

print("ℹ️ Для удаления файлов раскомментируйте нужные команды выше")

## 12. Очистка (опционально)

Освободите место, удалив промежуточные файлы: